In [11]:
# Import Statements
import pyrealsense2 as rs
import numpy as np
import cv2
from pyzbar import pyzbar
import math
import copy
import re
import time

In [12]:
class rotation_estimator:
    def __init__(self):
        self.theta = [0.0,0.0,0.0]
        self.alpha = 0.98
        self.first = True
        self.last_ts_gyro = 0.0
    
    # Function to calculate the change in angle of motion based on data from gyro
    # gyro_data - list of 3 elems
    # ts - float arrival time of curr gyro frame
    def process_gyro(self, gyro_data, ts):
        if self.first:
            self.last_ts_gyro = ts
            return
        
        gyro_angle = [gyro_data.x,gyro_data.y,gyro_data.z]
        
        dt_gyro = (ts-self.last_ts_gyro)/1000.0
        self.last_ts_gyro = ts
        
        gyro_angle = [gyro_angle[0]*dt_gyro,gyro_angle[1]*dt_gyro,gyro_angle[2]*dt_gyro]
        
        self.theta = [self.theta[0]-gyro_angle[2],self.theta[1]-gyro_angle[1],self.theta[2]+gyro_angle[0]]
    
    # Function to calculate the change in angle of motion based on data from accelerometer
    # accel_data - list of 3 elems
    def process_accel(self, accel_data):
        accel_angle = [0.0,0.0,0.0]
        accel_angle[2] = np.arctan2(accel_data.y,accel_data.z)
        accel_angle[0] = np.arctan2(accel_data.x, np.sqrt(accel_data.y**2+accel_data.z**2))
        
        if self.first:
            self.first = False
            self.theta = accel_angle
            self.theta[1] = np.pi/2
        else:
            self.theta[0] = self.theta[0]*self.alpha + accel_angle[0]*(1-self.alpha)
            self.theta[2] = self.theta[2]*self.alpha + accel_angle[2]*(1-self.alpha)
        
    def get_theta(self):
        theta_out = [0.0,0.0,0.0]
        theta_out = [self.theta[0]*180/np.pi, self.theta[1]*180/np.pi, (self.theta[2]-np.pi/2)*180/np.pi]
        return theta_out

In [13]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30) # depth stream
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30) # color stream
config.enable_stream(rs.stream.gyro) # gyro stream
config.enable_stream(rs.stream.accel) # accelerometer stream
align = rs.align(rs.stream.color) # align both streams to same pov

In [14]:
# Enable visualizer and filters for later use
colorizer = rs.colorizer()
spatial = rs.spatial_filter()
spatial.set_option(rs.option.filter_magnitude, 5)
spatial.set_option(rs.option.filter_smooth_alpha, 1)
spatial.set_option(rs.option.filter_smooth_delta, 50)
spatial.set_option(rs.option.holes_fill, 3)
hole_filling = rs.hole_filling_filter()
depth_to_disparity = rs.disparity_transform(True)
disparity_to_depth = rs.disparity_transform(False)

In [15]:
# Start streaming
algo = rotation_estimator()
profile = pipeline.start(config)
first_run = True

try:
    while True:        
        if first_run:
            try:
                frames = pipeline.wait_for_frames()
                
                accel_frame = frames.first_or_default(rs.stream.accel)
                if accel_frame:
                    accel_data = accel_frame.as_motion_frame().get_motion_data()
                
                gyro_frame = frames.first_or_default(rs.stream.gyro)
                if gyro_frame:
                    gyro_data = gyro_frame.as_motion_frame().get_motion_data()
                    gyro_ts = gyro_frame.as_motion_frame().get_timestamp()
            
                if not accel_frame or not gyro_frame:
#                     print("FIRST NOT FOUND")
                    break
                    
                algo.process_gyro(gyro_data, gyro_ts)
                algo.process_accel(accel_data)
                first_run = False
#                 print("FIRST PROCESSED")
                continue

            except:
#                 print("EXCEPTION")
                break
        
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        frames = align.process(frames)
        
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        accel_frame = frames.first_or_default(rs.stream.accel)
        if accel_frame:
            accel_data = accel_frame.as_motion_frame().get_motion_data()

        gyro_frame = frames.first_or_default(rs.stream.gyro)
        if gyro_frame:
            gyro_data = gyro_frame.as_motion_frame().get_motion_data()
            gyro_ts = gyro_frame.as_motion_frame().get_timestamp()

        if not accel_frame or not gyro_frame:
#             print("LOOPED ACCEL/GYRO FRAME NOT FOUND")
            continue

        algo.process_gyro(gyro_data, gyro_ts)
        algo.process_accel(accel_data)

        if not depth_frame or not color_frame:
#             print("LOOPED DEPTH/COLOR FRAME NOT FOUND")
            continue

        # filter depth stream: depth2disparity -> spatial -> disparity2depth -> hole_filling
        depth_frame = depth_to_disparity.process(depth_frame)
        depth_frame = spatial.process(depth_frame)
        depth_frame = disparity_to_depth.process(depth_frame)
        depth_frame = hole_filling.process(depth_frame)

        # get intrinsics
        depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
        color_intrin = color_frame.profile.as_video_stream_profile().intrinsics
        
        # Convert images to numpy arrays
        depth_image = np.asanyarray(colorizer.colorize(depth_frame).get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        # detect and decode barcodes
        object_depth = np.asanyarray(depth_frame.get_data())
        barcode_poses = []
        barcodes = pyzbar.decode(color_image)
            
        bar_x = []
        bar_y = []
        robo_x = []
        robo_y = []
        
        for barcode in barcodes:
            (x, y, w, h) = barcode.rect
            cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.rectangle(depth_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            barcodeData = barcode.data.decode("utf-8")
            barcodeType = barcode.type
            
            curr = copy.deepcopy(object_depth)
            curr = curr[math.floor(((x)+(x+w))/2-1):math.ceil(((x)+(x+w))/2+1),math.floor(((y)+(y+h))/2-1):math.ceil(((y)+(y+h))/2+1)].astype(float)
            depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
            curr = curr * depth_scale
            dist,_,_,_ = cv2.mean(curr)
            barcode_poses.append((barcodeData, dist))
 
            # draw the barcode data and barcode type on the image
            text = "{} ({})".format(barcodeData, dist)
            cv2.putText(color_image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            # store poses of codes
            curr_pose = re.split(',|\(|\)',barcodeData)
            bar_x.append(float(curr_pose[1]))
            bar_y.append(float(curr_pose[2]))
            
            # Get 3d point of object detected
            relative_depth_point = rs.rs2_deproject_pixel_to_point(depth_intrin, [int((x+(x+w))/2), int((y+(y+h))/2)], dist)
#             print("\Relative Point: \nX ",relative_depth_point[0],"\nY ",relative_depth_point[1],"\nZ ",relative_depth_point[2])
            
            
            # Get world position of cam according to this barcode
            curr_cam_theta = algo.get_theta()[1]
#             print("Cam Theta: \n",curr_cam_theta)
            curr_cam_phi = np.arctan2(relative_depth_point[2],relative_depth_point[0])*180/np.pi
#             print("Cam Phi: \n",curr_cam_phi)
            curr_cam_alpha = 90 - curr_cam_theta + curr_cam_phi
#             print("Cam Alpha: \n",curr_cam_alpha)
            curr_cam_delta_x = dist*np.cos(np.radians(curr_cam_alpha))
#             print("Cam delta_x: \n",curr_cam_delta_x)
            curr_cam_delta_y = dist*np.sin(np.radians(curr_cam_alpha))
#             print("Cam delta_y: \n",curr_cam_delta_y)             
#             print("\nBarcode Pose: \nX ",curr_pose[1],"\nY ",curr_pose[2])
            robo_x.append(float(curr_pose[1])-float(curr_cam_delta_x))
            robo_y.append(float(curr_pose[2])-float(curr_cam_delta_y))

        if barcodes:
            curr_cam_pos = [np.average(robo_x), np.average(robo_y)]
            
            text_pos_x = 'X: ' + str(curr_cam_pos[0])
            text_pos_y = 'Y: ' + str(curr_cam_pos[1])
            
            cv2.putText(depth_image, text_pos_x, (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(depth_image, text_pos_y, (20, 420), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
        # Stack both images horizontally
        images = np.hstack((color_image, depth_image))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        key = cv2.waitKey(1)
        
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
        
finally:

    # Stop streaming
    pipeline.stop()